# Train SVM File
***

1. Import libraries
2. Load preprocessed data
3. Train SVM
4. Save model

In [12]:
# 1. import libraries
import numpy as np
import pandas as pd
import torch
# import svm classifier from sklearn
from sklearn.svm import SVC


In [4]:
# constants

CV = 1

In [34]:
# list of test accuracies
test_acc = []
path = '../../Preprocessed Data/'
for iteration in range(CV) :
    
    print("K-Fold Cross Validation Iteration: ", iteration)

    # load data from numpy array
    train = np.load(path +'X_train_' + str(iteration+1) + '.npy', allow_pickle=True)
    # X_train = np.load('X_train_' + str(iteration) + '.npy')
    # y_train = np.load('y_train_' + str(iteration) + '.npy')
    traindf = pd.DataFrame(train, columns = ["Tensor", "Target"])
    X_train = traindf.Tensor
    X_train = tuple(X_train.to_list())
    X_train = torch.stack(X_train)
    y_train = traindf.Target.array
    val = np.load(path +'X_val_' + str(iteration+1) + '.npy', allow_pickle=True)
    test = np.load(path +'X_test_' + str(iteration+1) + '.npy', allow_pickle=True)
    X_val = val[0]
    y_val = val[1]
    X_test = test[0]
    y_test = test[1]
    # X_val = np.load('X_val_' + str(iteration) + '.npy')
    # y_val = np.load('y_val_' + str(iteration) + '.npy')
    # X_test = np.load('X_test_' + str(iteration) + '.npy')
    # y_test = np.load('y_test_' + str(iteration) + '.npy')

    # tune the hyperparameters
    # 1. Kernel type
    # 2. C
    kernels = ['linear', 'poly', 'rbf', 'sigmoid']
    C = np.logspace(-3, 3)


    val_acc = np.zeros((len(kernels), len(C)))


    for kernel in kernels :
        for c in C :
            # create a svm classifier
            svm = SVC(kernel=kernel, C=c)
            # fit the model
            svm.fit(X_train, y_train)
            # predict the model
            y_pred = svm.predict(X_val)
            # calculate the accuracy
            accuracy = np.sum(y_pred == y_val) / len(y_val)
            # append the validation accuracy to the list
            val_acc[kernels.index(kernel), C.index(c)] = accuracy

    # get the best hyperparameters
    best_kernel = kernels[np.argmax(val_acc) // len(C)]
    best_C = C[np.argmax(val_acc) % len(C)]

    # create a svm classifier with the best hyperparameters
    svm = SVC(kernel=best_kernel, C=best_C)

    # fit the model with the training data
    svm.fit(X_train, y_train)

    # predict the model with the test data
    y_pred = svm.predict(X_test)


    # calculate the accuracy
    accuracy = np.sum(y_pred == y_test) / len(y_test)

    print("Fold", iteration, "Test Accuracy: ", accuracy)

    # append the test accuracy to the list
    test_acc.append(accuracy)


print('The average test accuracy is: ', np.mean(test_acc))


K-Fold Cross Validation Iteration:  0


ValueError: Found array with dim 4. SVC expected <= 2.

In [38]:
traindf = pd.DataFrame(train, columns = ["Tensor", "Target"])
X_train = traindf.Tensor
y_train = traindf.Target


In [39]:
X_train = tuple(X_train.to_list())
X_train = torch.stack(X_train)

In [40]:
X_train.shape

torch.Size([1600, 2, 64, 430])

In [45]:
X_train[0][0][0][0]

tensor(3.9960)